In [1]:
import os
import numpy as np
from tifffile import imwrite, imread
import napari
import pickle

### Pathways

In [2]:
dir_movies  = r'R:\Aidan\240910_tiff_movies'
movie_name = 'B3 Region1_Merged'
channels = [0,1]
channel_colors = ['gray','red'] # has to match the number of channels
contrast_limits = [[0, 70], [0, 200]] # has to match the number of channels

dir_save = r'R:\Aidan\240919_separate_organoid_movies'

USE_SMALL = True
SMALL_FACTOR = 4

BIT_DEPTH = np.uint16

### Load and display movies

In [4]:
viewer = napari.Viewer()

for ch,ch_color,contr in zip(channels,channel_colors, contrast_limits):

    if USE_SMALL == True:
        channel_path = os.path.join(dir_movies, movie_name + f'_ch_{str(ch).zfill(2)}_small.tif')
    else:
        channel_path = os.path.join(dir_movies, movie_name + f'_ch_{str(ch).zfill(2)}.tif')

    temp = imread(channel_path)
    viewer.add_image(temp, name=f'ch{ch}', colormap=ch_color, blending = 'additive', contrast_limits=contr)

viewer.add_shapes()

<Shapes layer 'Shapes' at 0x2b595793e50>

###################################################
## Manual part - mark all organoids with rectangles
###################################################

### Save individual movies

In [5]:
# save positions of the organoids
shapes_layer = viewer.layers['Shapes']

shapes_path = os.path.join(dir_save, movie_name + '_shapes.pkl')
with open(shapes_path, 'wb') as file:
    pickle.dump(shapes_layer.data, file)

# create and save visualization of the organoids positions
shapes_layer.opacity = 0.7

shapes_layer.text = {
    'string': [f'Organoid {i}' for i in range(len(shapes_layer.data))],
    'size': 12,
    'color': 'black',
    'anchor': 'center',
}

visual_path = os.path.join(dir_save, movie_name + '_shapes_visual.png')
viewer.reset_view()
screenshot = viewer.screenshot(visual_path)

# save individual movies
for ch,ch_color,contr in zip(channels,channel_colors, contrast_limits):

    channel_path = os.path.join(dir_movies, movie_name + f'_ch_{str(ch).zfill(2)}.tif')

    temp = imread(channel_path)

    for ind,rect in enumerate(shapes_layer.data):
         
        if USE_SMALL == False:
             SMALL_FACTOR = 1

        # rescale position data
        rect = (rect * SMALL_FACTOR).astype(int)

        # cut out the organoid
        org_ch = temp[:,np.min(rect[:,0]):np.max(rect[:,0]), np.min(rect[:,1]):np.max(rect[:,1])]
                      
        # save the organoid
        save_path = os.path.join(dir_save, movie_name + f'_ch_{str(ch).zfill(2)}_organoid_{str(ind).zfill(2)}.tif')
        imwrite(save_path, org_ch.astype(BIT_DEPTH))
